In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import InceptionV3
from keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Load the Dogs vs. Cats dataset
train_dataset, info = tfds.load('cats_vs_dogs', split='train[:80%]', with_info=True, as_supervised=True)
test_dataset = tfds.load('cats_vs_dogs', split='train[80%:]', as_supervised=True)


# Preprocess the data
IMG_SIZE = 299

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    return image, label

train_dataset = train_dataset.map(preprocess).shuffle(1024).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
# Show some images with their labels
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow((images[i].numpy() + 1) / 2)
        plt.title("Cat" if labels[i].numpy() == 0 else "Dog")
        plt.axis('off')
plt.show()


In [ ]:
# Load the Inception-v3 model
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
print(inception_model.summary())


In [ ]:
# Create your model
x = inception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inception_model.input, outputs=predictions)

# Freeze the pretrained part
for layer in inception_model.layers:
    layer.trainable = False

# Print model summary
print(model.summary())


In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())


In [ ]:
# Train your model
history = model.fit(train_dataset, epochs=5, validation_data=test_dataset)


In [ ]:
#report the accuracy on your test set
##your code goes here##